In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Get columns name of all data set


In [ ]:
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')

bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')

prev = pd.read_csv('previous_application.csv')
pos = pd.read_csv('POS_CASH_balance.csv')
installments = pd.read_csv('installments_payments.csv')
credit = pd.read_csv('credit_card_balance.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'application_train.csv'

In [ ]:
print("application_train:", application_train.shape)
print(application_train.columns.tolist())
print("application_test:", application_test.shape)
print(application_test.columns.tolist())
print("bureau:", bureau.shape)
print(bureau.columns.tolist())
print("bureau_balance:", bureau_balance.shape)
print(bureau_balance.columns.tolist())
print("previous_application:", prev.shape)
print(prev.columns.tolist())
print("POS_CASH_balance:", pos.shape)
print(pos.columns.tolist())
print("installments_payments:", installments.shape)
print(installments.columns.tolist())
print("credit_card_balance:", credit.shape)
print(credit.columns.tolist())


In [ ]:
print(application_train.shape)
print(application_test.shape)

Right now merge dataset then basic EDA and cleaning

Merge bureau_balance into bureau, then aggregate to SK_ID_CURR

In [ ]:
bb_encoded = pd.get_dummies(bureau_balance, columns=['STATUS'])

bb_agg = bb_encoded.groupby('SK_ID_BUREAU').agg('mean').reset_index()

bureau_merged = bureau.merge(bb_agg, on='SK_ID_BUREAU', how='left')

numeric_cols = bureau_merged.select_dtypes(include=['number']).columns.tolist()
if 'SK_ID_CURR' not in numeric_cols:
    numeric_cols.append('SK_ID_CURR')

bureau_numeric = bureau_merged[numeric_cols]


bureau_agg = bureau_numeric.groupby('SK_ID_CURR').agg(['mean', 'max', 'min', 'sum'])

bureau_agg.columns = ['BUREAU_' + '_'.join(col).upper() for col in bureau_agg.columns]
bureau_agg.reset_index(inplace=True)


Merge previous_application with child tables

In [ ]:
def agg_child(df, name):
    numeric_df = df.select_dtypes(include=['number']).copy()
    numeric_df['SK_ID_PREV'] = df['SK_ID_PREV']

    df_agg = numeric_df.groupby('SK_ID_PREV').agg(['mean'])
    df_agg.columns = [f"{name}_" + "_".join(col).upper() for col in df_agg.columns]
    return df_agg.reset_index()

pos_agg = agg_child(pos, "POS")
inst_agg = agg_child(installments, "INST")
credit_agg = agg_child(credit, "CREDIT")

prev = prev.merge(pos_agg, on='SK_ID_PREV', how='left')
prev = prev.merge(inst_agg, on='SK_ID_PREV', how='left')
prev = prev.merge(credit_agg, on='SK_ID_PREV', how='left')

prev_numeric = prev.select_dtypes(include=['number']).copy()

if 'SK_ID_CURR' not in prev_numeric.columns:
    prev_numeric['SK_ID_CURR'] = prev['SK_ID_CURR']

prev_numeric = prev_numeric.drop(columns=['SK_ID_PREV'], errors='ignore')

prev_agg = prev_numeric.groupby('SK_ID_CURR').agg(['mean'])
prev_agg.columns = ['PREV_' + '_'.join(col).upper() for col in prev_agg.columns]
prev_agg.reset_index(inplace=True)

print("✅ prev_agg shape:", prev_agg.shape)


In [ ]:
application_train = application_train.merge(prev_agg, on='SK_ID_CURR', how='left')

application_test = application_test.merge(prev_agg, on='SK_ID_CURR', how='left')

print("✅ Train shape after merge:", application_train.shape)
print("✅ Test shape after merge:", application_test.shape)


In [ ]:
application_train = application_train.merge(bureau_agg, on='SK_ID_CURR', how='left')
application_test = application_test.merge(bureau_agg, on='SK_ID_CURR', how='left')

print("✅ Final Train shape:", application_train.shape)
print("✅ Final Test shape:", application_test.shape)


save file final merger

In [ ]:
train_path = "application_train_merged.csv"
test_path = "application_test_merged.csv"

application_train.to_csv(train_path, index=False)
application_test.to_csv(test_path, index=False)

we complete merge next basic EDA on application_train

In [ ]:
missing_values = application_train.isnull().mean().sort_values(ascending=False) * 100
missing_values = missing_values[missing_values > 0]

print("Columns with missing values (%):")
display(missing_values.to_frame('MissingPercent').head(30))  # Top 30


In [ ]:
application_train.info()


In [ ]:
application_train.describe().T.sort_values(by='std', ascending=False).head(20)



In [ ]:
categorical_cols = application_train.select_dtypes(include=['object']).columns.tolist()

for col in categorical_cols:
    print(f"{col}: {application_train[col].nunique()} unique values")


In [ ]:
sns.countplot(x='TARGET', data=application_train)
plt.title('Target Variable Distribution (0: Not Default, 1: Default)')
plt.show()

print(application_train['TARGET'].value_counts(normalize=True) * 100)


Handle Missing Values (Without Losing Important Features)

In [ ]:
missing = application_train.isnull().mean().sort_values(ascending=False)

threshold = 0.4
drop_cols = missing[missing > threshold].index.tolist()

application_train_clean = application_train.drop(columns=drop_cols)
application_test_clean = application_test.drop(columns=drop_cols)

print(f"Dropped {len(drop_cols)} columns with >{threshold*100}% missing values")
